In [34]:
%load_ext autoreload
%autoreload 2

import docker
import git
import os
import yaml

from defaults import motion_list, regressor_list
from subjects import generate_subject_list_for_range

with open('../configs/cpac.yml', 'r') as config_file:
    config_settings = yaml.safe_load(config_file)
    
cpac_dir = os.path.abspath(os.path.join(os.getcwd(), *[os.pardir for i in range(3)]))    
n_cpus = 10
pipeline_file = os.path.join(cpac_dir, 'dev/docker_data/default_pipeline.yml')
tag = git.Repo(search_parent_directories=True).head.object.hexsha[:7]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
client = docker.from_env()

client.images.build(path=cpac_dir, tag=tag)

BuildError: The command '/bin/sh -c apt-get install -y --no-install-recommends       fsl-core       fsl-atlases       fsl-mni152-templates' returned a non-zero code: 100

In [33]:
display(" ".join([
    f"docker run -v {config_settings['this_run_outputs']}:/outputs "
    f"-v {config_settings['data_path']}:/bids_dataset "
    f"-v /data3/cnl/fmriprep/cpac_run/configs:/configs fcpindi/c-pac:{tag} "
    f"/bids_dataset /outputs participant --pipeline_file {pipeline_file} --n_cpus {n_cpus} --participant_label",
    *generate_subject_list_for_range(
        (config_settings['subjects']['start'], config_settings['subjects']['stop'])
    )
]))

'docker run -v /data3/cnl/auto-heatmaps/cpac_default:/outputs -v /data3/cnl/fmriprep/HNUdata/HNU30s:/bids_dataset -v /data3/cnl/fmriprep/cpac_run/configs:/configs fcpindi/c-pac:3a47f04 /bids_dataset /outputs participant --pipeline_file /home/jclucas/C-PAC/dev/docker_data/default_pipeline.yml --n_cpus 10 --participant_label sub-0025427 sub-0025428 sub-0025429 sub-0025430 sub-0025431 sub-0025432 sub-0025433 sub-0025434 sub-0025435 sub-0025436 sub-0025437 sub-0025438 sub-0025439 sub-0025440 sub-0025441 sub-0025442 sub-0025443 sub-0025444 sub-0025445 sub-0025446 sub-0025447 sub-0025448 sub-0025449 sub-0025450 sub-0025451 sub-0025452 sub-0025453 sub-0025454 sub-0025455 sub-0025456'